<a href="https://colab.research.google.com/github/bustanularifin666/MachineLearningTasks/blob/main/TaskWeek13/CNN_Modern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Muhammad Bustanul Arifin Nur Fathurrahman
1103228224

# 1. AlexNet

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST dataset (images and labels)
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize to fit AlexNet input dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader (input pipeline)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Define the AlexNet architecture
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Rest of the code remains the same for training and evaluation.

# Rest of the code remains the same for training and evaluation.

# Rest of the code remains the same for training and evaluation.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet().to(device)

# 3. Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training and Evaluation
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss:.4f}")

def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    corrects = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)

        test_loss = running_loss / len(test_loader.dataset)
        accuracy = corrects.double() / len(test_loader.dataset)
        print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, epochs=10)

# Evaluate the model
evaluate(model, test_loader, criterion)


Epoch 1/10, Training Loss: 0.2555
Epoch 2/10, Training Loss: 0.0903
Epoch 3/10, Training Loss: 0.0793
Epoch 4/10, Training Loss: 0.0649
Epoch 5/10, Training Loss: 0.0693
Epoch 6/10, Training Loss: 0.0554
Epoch 7/10, Training Loss: 0.0511
Epoch 8/10, Training Loss: 0.0515
Epoch 9/10, Training Loss: 0.0488
Epoch 10/10, Training Loss: 0.0466
Test Loss: 0.0386, Accuracy: 0.9903


- import torch: Mengimpor library utama PyTorch.

- import torch.nn as nn: Mengimpor modul yang berisi kelas-kelas untuk mendefinisikan dan membangun arsitektur neural network. nn merupakan singkatan dari "neural network."

- import torch.optim as optim: Mengimpor modul yang berisi implementasi berbagai algoritma optimasi untuk melatih neural network.

- import torchvision: Library yang menyediakan dataset populer, arsitektur model, dan alat bantu lainnya untuk tugas computer vision.

- import torchvision.transforms as transforms: Modul yang berisi fungsi-fungsi untuk melakukan transformasi pada data gambar, seperti normalisasi, cropping, dan flipping.

- from torch.utils.data import DataLoader: Mengimpor kelas DataLoader dari modul torch.utils.data. DataLoader digunakan untuk memuat data dalam batch ke dalam model saat melatih atau menguji.

# 2. LeNet 5

In [7]:
# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.Resize((32, 32)), # For LeNet
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct


In [10]:
# Example for LeNet-5
model = LeNet5().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, correct = test(model, device, test_loader, criterion)
    print(f"LeNet-5 - Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


LeNet-5 - Epoch 1: Test Loss: 0.0012, Accuracy: 9755/10000
LeNet-5 - Epoch 2: Test Loss: 0.0008, Accuracy: 9841/10000
LeNet-5 - Epoch 3: Test Loss: 0.0006, Accuracy: 9874/10000
LeNet-5 - Epoch 4: Test Loss: 0.0005, Accuracy: 9900/10000
LeNet-5 - Epoch 5: Test Loss: 0.0006, Accuracy: 9893/10000
LeNet-5 - Epoch 6: Test Loss: 0.0006, Accuracy: 9876/10000
LeNet-5 - Epoch 7: Test Loss: 0.0005, Accuracy: 9904/10000
LeNet-5 - Epoch 8: Test Loss: 0.0005, Accuracy: 9883/10000
LeNet-5 - Epoch 9: Test Loss: 0.0006, Accuracy: 9877/10000
LeNet-5 - Epoch 10: Test Loss: 0.0006, Accuracy: 9891/10000


# 3. VGGNet

In [11]:
# MNIST dataset (images and labels)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit VGGNet input dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader (input pipeline)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [12]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Additional convolutional layers can be added here
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(128 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [13]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


In [14]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct


In [15]:
model = VGGNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):  # 10 epochs
    train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, correct = test(model, device, test_loader, criterion)
    print(f"Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


Epoch 1: Test Loss: 0.0009, Accuracy: 9811/10000
Epoch 2: Test Loss: 0.0007, Accuracy: 9860/10000
Epoch 3: Test Loss: 0.0006, Accuracy: 9876/10000
Epoch 4: Test Loss: 0.0006, Accuracy: 9882/10000
Epoch 5: Test Loss: 0.0006, Accuracy: 9895/10000
Epoch 6: Test Loss: 0.0006, Accuracy: 9885/10000
Epoch 7: Test Loss: 0.0005, Accuracy: 9902/10000
Epoch 8: Test Loss: 0.0006, Accuracy: 9889/10000
Epoch 9: Test Loss: 0.0007, Accuracy: 9887/10000
Epoch 10: Test Loss: 0.0008, Accuracy: 9875/10000
